In [1]:
from collections import defaultdict
from pymongo import MongoClient
from tqdm import tqdm

# client = MongoClient("mongodb://10.0.52.71:27017/")
# db = client["colabfit-2023-5-16"]

client = MongoClient("mongodb://localhost:27017/")
db = client["mp"]


makelist unequal_md_do_relationships
for each data object
    get relevant PIs
    if relationship is 1(MD)->1(DO):
        find DO-id position in array
        find corresponding MD in array
        DO.update with field "relationships.metadata(array)"
    elif relationship is 1(MD)->many(DO):
        DO.update with field "relationships.metadata(array)"
    else:
        append DO-id to unequal_md_do_relationships (to examine later)

    

In [ ]:
db.property_instances.find(
    {
        "$and": [
            {"relationships.data_objects.1": {"$exists": true}},
            {"relationships.metadata.1": {"$exists": true}},
            {
                {"$size": "relationships.metadata"}: {
                    "$not": {"$size": "relationships.data_objects"}
                }
            },
        ]
    }
).count()
{{"$size": "relationships.metadata"}: {"$not": {"$size": "relationships.data_objects"}}}

db.property_instances.aggregate([
  {
    $match:
      {
        "relationships.metadata.1": {
          $exists: true,
        },
        "relationships.data_objects.1": {
          $exists: true,
        },
      },
  },
  {
    $project:
      {
        mds: {
          $size: "$relationships.metadata",
        },
        dos: {
          $size: "$relationships.data_objects",
        },
      },
  },
  {
    $match:
      {
        mds: {
          $ne: "dos",
        },
      },
  },
  {
    $count:
      "num_mds_ne_dos",
  },
])